# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [612]:
# import libraries
#for wrangling
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
#for regular expressions
import re
#for natural language processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
#for modelling
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
#for pickeling
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/philipherp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philipherp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [464]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_final', engine)
#as specified in part 3
X = df['message'].values
Y = df.iloc[:,4:].values
cat = df.iloc[:, 4:].columns.tolist()

In [352]:
print('x shape: ', X.shape)
print('y shape: ', Y.shape)

x shape:  (26216,)
y shape:  (26216, 36)


In [278]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [269]:
def tokenize(text):
    """
    Normalizes text: Removes punctuation, tokenizes text, cleans tokens, makes al lowercase and removes spaces, lematizes tokens
    Args:
        - text: for countVectorizer from sklearn (Convert a collection of text documents to a matrix of token counts)
    Returns:
        - clean_tokens
    """
    # remove punctuation characters
    text = re.sub(r"[^A-Za-z0-9\-]", " ", text)
    
    #creates tokens
    tokens = word_tokenize(text)
    
    #convert token to meaningful base form
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        #actually lemmatize, make all lowercase, remove space and append
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#choosing the right estimator
#need to choose estimator for multiclass-output --> https://scikit-learn.org/stable/modules/multiclass.html?highlight=multioutputclassifier

In [587]:
#build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3, leaf_size=30, n_jobs=-1)))    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [588]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

#train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fc881280a70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1,
                                                                      n_neighbors=3)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [589]:
#predict on test data (X_test)
y_pred = pipeline.predict(X_test)

In [620]:
#print f1 score, precision
for i in range(len(cat)):
    print(classification_report(y_test[:, i], y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.59      0.30      0.39      2054
           1       0.80      0.93      0.86      6534
           2       0.31      0.33      0.32        64

    accuracy                           0.78      8652
   macro avg       0.57      0.52      0.53      8652
weighted avg       0.75      0.78      0.75      8652

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      7180
           1       0.86      0.06      0.12      1472

    accuracy                           0.84      8652
   macro avg       0.85      0.53      0.51      8652
weighted avg       0.84      0.84      0.78      8652

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0

### 6. Improve your model
Use grid search to find better parameters. 

In [591]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fc881280a70>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1, n_neighbors=3)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fc881280a70>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1, n_neighbors=3)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf'

In [608]:
#gridsearch parameters
parameters = {
    'clf__estimator__n_neighbors': [5],
    'clf__estimator__leaf_size': [30]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [609]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fc881280a70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1,
                                                                                             n_neighbors=3)))]),
             param_grid={'clf__estimator__leaf_size': [30],
                         'clf__estimator__n_neighbors': [5]})

In [610]:
print(cv.best_score_, cv.best_params_)

0.2459018957896054 {'clf__estimator__leaf_size': 30, 'clf__estimator__n_neighbors': 5}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [618]:
y_pred=cv.predict(X_test)

In [619]:
for i in range(len(cat)):
    print(classification_report(y_test[:, i], y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.59      0.30      0.39      2054
           1       0.80      0.93      0.86      6534
           2       0.31      0.33      0.32        64

    accuracy                           0.78      8652
   macro avg       0.57      0.52      0.53      8652
weighted avg       0.75      0.78      0.75      8652

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      7180
           1       0.86      0.06      0.12      1472

    accuracy                           0.84      8652
   macro avg       0.85      0.53      0.51      8652
weighted avg       0.84      0.84      0.78      8652

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [616]:
#with open('model.pkl', 'wb') as f:
#        pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.